In [16]:
import os

from os.path import isfile


def call_vienna_plfold(sequence, seq_name, W=200,L=150):
    '''Runs Vienna RNAfold with partition function for all sequences inside input fasta file 
    Input: sequence as string of nucelutides
           seq_name as string to be used for intermediate and output file names
    '''

    from subprocess import Popen, PIPE
    dp_file_name = "{}_dp.ps".format(seq_name)
    unp_file_name = "{}_lunp".format(seq_name)
    if isfile(dp_file_name): # Caution Race condition may occur 
        os.remove(dp_file_name)
    if isfile(unp_file_name): # Caution Race condition may occur 
        os.remove(unp_file_name)
    
    RNAPLFOLD = 'RNAplfold -W {} -L {} -u 1 '.format(W, L)  # -u 1 for unpaired probablitiy 
    print (sequence)
    assert len(sequence.split()) == 1
    cmd = ('echo ">%s\\n%s\\n" | '%(seq_name, sequence))  
    cmd += RNAPLFOLD 
    print(cmd)
    p = Popen(cmd, stdin=PIPE, shell=True, stdout=PIPE, stderr=PIPE)
    out, err = p.communicate()
    #print (out)
    #print (err)
    has_error = False
    if err:
        if b"warn" in err.lower():
            print ("Warning in calling call_RNAfold:\n {} {}\n".format(out, err))
        else:
            raise RuntimeError("Error in calling call_RNAfold: {} {}\n".format(out, err))
    if not isfile(dp_file_name):
        raise RuntimeError("Error: Expected dp file: {} is not created!".format(dp_file_name))
    if not isfile(unp_file_name):
        raise RuntimeError("Error: Expected lunp file: {} is not created!".format(unp_file_name))
    
    return dp_file_name, unp_file_name

def get_unpaired_probs(unp_file):
    '''
    Reads Vienna RNAplfold unpaired prob file (-u 1) into dict
    Returns: a dictionary with 1-based positions as key and
             unpaired-probabilities, aka accessibilities, as values
    '''
    
    with open(unp_file) as unp_in:
        line1 = unp_in.readline()
        if "#unpaired probabilities" not in line1:
            raise IOError('Unexpected header for lunp file: {}'.format(line1))
        line2 = unp_in.readline()
        if "#i$\tl=1" not in line2:
            raise IOError('Unexpected second header for lunp file: {}'.format(line2))
        up_dic = dict()
        for line in unp_in:
            splits = line.split()
            assert len(splits) >= 2
            pos = int(splits[0])
            up_prob = float(splits[1])
            assert pos >=1
            assert up_prob >= 0 and up_prob <= 1
            assert pos not in up_dic
            up_dic[pos] = up_prob
            
    
    return up_dic
dotplot_file, accessibility_file = call_vienna_plfold('ACGCGGCGAAGAGACGTTGATGAGCCTGCTTAGACCGATAACAC','seq1')
get_unpaired_probs(accessibility_file)

ACGCGGCGAAGAGACGTTGATGAGCCTGCTTAGACCGATAACAC
echo ">seq1\nACGCGGCGAAGAGACGTTGATGAGCCTGCTTAGACCGATAACAC\n" | RNAplfold -W 200 -L 150 -u 1 
Warning in calling call_RNAfold:
 >seq1




{1: 0.9931936,
 2: 0.9829167,
 3: 0.7019946,
 4: 0.03892923,
 5: 0.002802452,
 6: 0.006038695,
 7: 0.08151145,
 8: 0.958596,
 9: 0.5796458,
 10: 0.2710047,
 11: 0.2922246,
 12: 0.528187,
 13: 0.522384,
 14: 0.9571145,
 15: 0.7446448,
 16: 0.517658,
 17: 0.4455539,
 18: 0.4377715,
 19: 0.9662073,
 20: 0.9996754,
 21: 0.8698157,
 22: 0.9425477,
 23: 0.5625841,
 24: 0.05619108,
 25: 0.2848084,
 26: 0.339093,
 27: 0.5345217,
 28: 0.7231078,
 29: 0.06057903,
 30: 0.2991378,
 31: 0.5168907,
 32: 0.9632038,
 33: 0.3840509,
 34: 0.9996165,
 35: 0.3133471,
 36: 0.3122999,
 37: 0.3271267,
 38: 0.9986576,
 39: 0.9592363,
 40: 0.9762574,
 41: 0.9818607,
 42: 0.9883941,
 43: 0.99686,
 44: 0.9993398}